# 看護師離職率データセット作成（ETL完全版）

このノートブックは、手作業によるデータ作成と同等の品質（正確な数値、適切な単位）を自動化するためのものです。
特に**「沖縄県のデータ異常」「年収の単位変換」「家賃の民営借家指定」**などの重要ロジックが実装されています。

## 前提
- 元データ（CSV/Excel）が `../data/raw/` に配置されていること。
- 出力先は `../data/processed/` です。

In [87]:
import pandas as pd
from pathlib import Path
import os

# ==========================
# Step 1-1: 偵察 (Load & Peek)
# ==========================
# DE Comment: パス操作にはOS非依存の pathlib を採用し、保守性を高める
RAW_DIR = Path("../data/raw")
OUT_DIR = Path("../data/out")

# DE Comment: 冪等性（Idempotency）担保のため、出力ディレクトリがない場合は自動作成する
OUT_DIR.mkdir(parents=True, exist_ok=True)

# DE Comment: ソースファイルの定義
file_turnover = RAW_DIR / "日本看護協会_離職率_都道府県別_2023.csv"

# DE Comment: 生データの中身を確認（EncodingやHeader位置の特定）
# 失敗時は 'cp932' (Shift-JIS) を試すが、今回は 'utf-8-sig' で指定
try:
    df_peek = pd.read_csv(file_turnover, encoding="utf-8-sig")
    print(f"Data Loaded Successfully. Shape: {df_peek.shape}")
    display(df_peek.head())
except Exception as e:
    print(f"Load Error: {e}")

Data Loaded Successfully. Shape: (47, 4)


,prefecture,turnover_total,turnover_new_grad,turnover_experienced
0,北海道,11.5,5.9,16.6
1,青森県,8.6,10.7,16.7
2,岩手県,6.8,7.8,19.1
3,宮城県,9.1,7.1,12.4
4,秋田県,7.4,5.0,7.3


In [88]:
import re

# ==========================
# Step 1-2: 加工と検問 (Transform & Validate)
# ==========================

def normalize_prefecture(x):
    """
    Data Cleansing: 表記揺れのある都道府県名を正規化し、JIS準拠のキーに変換する
    """
    # Null Handling: 欠損値は後続のフィルタリングで処理するためスルー
    if pd.isna(x):
        return x

    x = str(x)
    
    # Cleansing: トリム処理（全角・半角スペースの除去）
    x = re.sub(r"\s+", "", x)

    # Filtering: 集計行（全国計・総数など）や不適格レコードを除外対象とする
    if x in ["全国", "計", "未回答", "無回答・不明"]:
        return None

    # Exception Handling: 北海道は接尾辞ロジックの例外として処理
    if x == "北海道":
        return x

    # Normalization: 省略された接尾辞（都・府）を補完
    if x == "東京":
        return "東京都"
    if x == "京都":
        return "京都府"
    if x == "大阪":
        return "大阪府"

    # Normalization: 接尾辞がない場合は「県」をデフォルト付与
    if not re.search(r"[都道府県]$", x):
        x += "県"

    return x

# DE Comment: 本番読み込み（Ingest）
df_turnover = pd.read_csv(file_turnover, encoding="utf-8-sig")

# DE Comment: 分析用カラム名へのマッピング（Schema Mapping）
# 可読性とアクセス性を向上させるため、英語スネークケースにリネーム
rename_map = {
    '都道府県': 'prefecture',
    '正規雇用看護職員離職率_総数': 'turnover_total',
    '新卒看護職員離職率': 'turnover_new_grad',
    '既卒看護職員離職率': 'turnover_experienced'
}

# DE Comment: 必要な列のみを抽出しつつリネーム（Vertical Slicing）
df_master = df_turnover.rename(columns=rename_map)[list(rename_map.values())].copy()

# DE Comment: 都道府県名の正規化関数を適用（Key Standardization）
df_master["prefecture"] = df_master["prefecture"].apply(normalize_prefecture)

# DE Comment: 正規化の結果、キーが特定できなかった行（全国計など）を物理削除
df_master = df_master.dropna(subset=["prefecture"])

# --------------------------
# Data Quality Check (DQ)
# --------------------------
print("--- Data Quality Check (Step 1) ---")

# Assertion: 行数が47（都道府県数）であることを厳密に検証
# これがFailする場合は、全国計の除外漏れやデータの欠落を疑う
assert len(df_master) == 47, f"Critical Error: 行数が47ではありません (Actual: {len(df_master)})"

# Assertion: 主キー（prefecture）の一意性検証（Unique Key Constraint）
dup_count = df_master["prefecture"].duplicated().sum()
assert dup_count == 0, f"Critical Error: 都道府県名に重複があります (Count: {dup_count})"

# Assertion: 数値列の欠損チェック（Completeness）
null_check = df_master[['turnover_total', 'turnover_new_grad', 'turnover_experienced']].isnull().sum().sum()
assert null_check == 0, "Error: 離職率データに欠損があります"

# Assertion: レンジチェック（Validity）。離職率は0-100%の範囲内であるはず
for col in ['turnover_total', 'turnover_new_grad', 'turnover_experienced']:
    assert df_master[col].between(0, 100).all(), f"Error: {col} に異常値（0-100範囲外）が含まれています"

print("✅ Pass: 行数(47), ユニークキー, 欠損なし, レンジ正常")
display(df_master.head())

--- Data Quality Check (Step 1) ---
✅ Pass: 行数(47), ユニークキー, 欠損なし, レンジ正常


,prefecture,turnover_total,turnover_new_grad,turnover_experienced
0,北海道,11.5,5.9,16.6
1,青森県,8.6,10.7,16.7
2,岩手県,6.8,7.8,19.1
3,宮城県,9.1,7.1,12.4
4,秋田県,7.4,5.0,7.3


In [89]:
# ==========================
# Step 1-3: 保存 (Save Checkpoint)
# ==========================

# DE Comment: ファイル命名規則に従い、Step番号と内容を明記
# master_step{NN}_{content}.csv
save_filename = "master_step01_turnover.csv"
save_path = OUT_DIR / save_filename

# DE Comment: クリーニング済みデータを永続化（Persistence）
# index=False で不要なインデックス列の出力を防ぐ
df_master.to_csv(save_path, index=False, encoding='utf-8-sig')

print(f"🎉 Checkpoint Saved: {save_path}")
print(f"Master Data Initialized with {len(df_master)} rows.")

🎉 Checkpoint Saved: ../data/out/master_step01_turnover.csv
Master Data Initialized with 47 rows.


In [91]:
import pandas as pd
from pathlib import Path

# ==========================
# Step 2: 夜勤データ統合 (Night Shift Integration)
# ==========================

# ==========================
# Step 2-1: 偵察 (Load & Peek)
# ==========================
# DE Comment: パス定義（Stepごとに再定義し、単独実行性を担保）
RAW_DIR = Path("../data/raw")
target_file = "日本看護協会_夜勤72h超過率_都道府県別_2024.csv"

print(f"=== Peeking: {target_file} ===")

# DE Comment: 生データの中身を確認（EncodingやHeader位置の特定）
try:
    # 厚労省・看護協会系は cp932 (Shift-JIS) が多い
    df_peek = pd.read_csv(RAW_DIR / target_file, encoding='cp932')
    display(df_peek.head())
    print(f"Raw Shape: {df_peek.shape}")
    print(f"Columns: {df_peek.columns.tolist()}")
except Exception as e:
    print(f"Load Error: {e}")

=== Peeking: 日本看護協会_夜勤72h超過率_都道府県別_2024.csv ===
Load Error: 'cp932' codec can't decode byte 0xef in position 0: illegal multibyte sequence


In [96]:
import pandas as pd
import re
from pathlib import Path

# ==========================
# Step 2-2: 加工と検問 (Transform & Validate)
# ==========================
RAW_DIR = Path("../data/raw")
target_file = "日本看護協会_夜勤72h超過率_都道府県別_2024.csv"

# Helper: 正規化関数
def normalize_prefecture(x):
    if pd.isna(x): return x
    x = str(x)
    x = re.sub(r"\s+", "", x)
    if x in ["全国", "計", "未回答", "無回答・不明"]: return None
    if x == "北海道": return x
    if x == "東京": return "東京都"
    if x == "京都": return "京都府"
    if x == "大阪": return "大阪府"
    if not re.search(r"[都道府県]$", x): x += "県"
    return x

# 1. Ingest (読み込み)
# DE Comment: BOM付きUTF-8として読み込み
df_night = pd.read_csv(RAW_DIR / target_file, encoding="utf-8-sig")

# Debug: 列名の確認
print("Loaded Columns:", df_night.columns.tolist())

# 2. Column Selection (列選択)
# DE Comment: 既に英語列名になっているため、リネームせずに必要な列を定義して抽出
target_cols = [
    'prefecture',
    'night_shift_72h_plus',
    'night_shifts_per_month_three_shift',
    'night_shifts_per_month_two_shift'
]

# カラム存在チェック
missing_cols = set(target_cols) - set(df_night.columns)
if missing_cols:
    raise ValueError(f"Critical Error: 以下の列が見つかりません。CSVを確認してください: {missing_cols}")

# 必要な列だけを抽出（Vertical Slicing）
df_night = df_night[target_cols].copy()

# 3. Cleansing (Key Normalization)
df_night['prefecture'] = df_night['prefecture'].apply(normalize_prefecture)
df_night = df_night.dropna(subset=['prefecture'])

# 4. DQ Check (Quality Assurance)
print("--- Data Quality Check (Step 2) ---")

# Assertion: 行数が47であること
assert len(df_night) == 47, f"Critical Error: 行数が47ではありません ({len(df_night)})"

# Assertion: キー重複なし
assert df_night['prefecture'].is_unique, "Critical Error: 都道府県名に重複があります"

# Assertion: 欠損なし
night_cols = [c for c in target_cols if c != 'prefecture']
assert df_night[night_cols].notnull().all().all(), "Error: 夜勤データに欠損があります"

print("✅ Pass: Clean Data Ready for Merge")
display(df_night.head())

Loaded Columns: ['prefecture', 'night_shift_72h_plus', 'night_shifts_per_month_three_shift', 'night_shifts_per_month_two_shift']
--- Data Quality Check (Step 2) ---
✅ Pass: Clean Data Ready for Merge


,prefecture,night_shift_72h_plus,night_shifts_per_month_three_shift,night_shifts_per_month_two_shift
0,北海道,36.7,7.8,4.6
1,青森県,36.5,7.7,4.8
2,岩手県,11.8,7.5,4.1
3,宮城県,30.2,8.0,4.7
4,秋田県,25.1,7.7,4.3


In [97]:
# ==========================
# Step 2-3: 結合と保存 (Merge & Save)
# ==========================
OUT_DIR = Path("../data/out")

# 1. Previous Load: 前のステップ（Step 01）の成果物をロード
prev_file = OUT_DIR / "master_step01_turnover.csv"
df_master = pd.read_csv(prev_file)

# 2. Idempotency (冪等性の確保)
# DE Comment: 既に同名の列がある場合、mergeで _x, _y が発生するのを防ぐため事前にDropする
# これにより「何度実行しても結果が同じ」状態を保てる
new_cols = [c for c in df_night.columns if c != 'prefecture']
df_master = df_master.drop(columns=new_cols, errors="ignore")

# 3. Merge (Left Join)
# DE Comment: Masterを左側にして、行数（47）を維持したまま結合
before_rows = len(df_master)
df_master = pd.merge(df_master, df_night, on='prefecture', how='left')
after_rows = len(df_master)

# 4. Post-Merge DQ (結合後チェック)
print("--- Merge Quality Check ---")
# Cartesian Product（行増殖）の検知
assert before_rows == after_rows, f"Critical Error: 結合前後で行数が変わりました ({before_rows} -> {after_rows})"
# 結合キー不一致による欠損発生の検知
null_check = df_master[new_cols].isnull().sum().sum()
if null_check > 0:
    missing_pref = df_master[df_master[new_cols].isnull().any(axis=1)]['prefecture'].tolist()
    print(f"⚠️ Warning: 結合できなかった都道府県があります: {missing_pref}")
    # 必要ならここで stop させる: assert null_check == 0
else:
    print("✅ All rows merged successfully.")

# 5. Checkpoint Save (保存)
# 命名規則: master_step{NN}_{content}.csv
save_filename = "master_step02_night_shift.csv"
save_path = OUT_DIR / save_filename
df_master.to_csv(save_path, index=False, encoding='utf-8-sig')

print(f"🎉 Checkpoint Saved: {save_path}")
print(f"Current Columns: {len(df_master.columns)}")
display(df_master.head())

--- Merge Quality Check ---
✅ All rows merged successfully.
🎉 Checkpoint Saved: ../data/out/master_step02_night_shift.csv
Current Columns: 7


,prefecture,turnover_total,turnover_new_grad,turnover_experienced,night_shift_72h_plus,night_shifts_per_month_three_shift,night_shifts_per_month_two_shift
0,北海道,11.5,5.9,16.6,36.7,7.8,4.6
1,青森県,8.6,10.7,16.7,36.5,7.7,4.8
2,岩手県,6.8,7.8,19.1,11.8,7.5,4.1
3,宮城県,9.1,7.1,12.4,30.2,8.0,4.7
4,秋田県,7.4,5.0,7.3,25.1,7.7,4.3
